Item 19: Never Unpack More Than Three Variables When Functions Return Multiple Values

1. Multiple return values from a function can also be unpacked by catch-all starred expression.
2. Unpacking into four or more variables is error prone and should be avoided; instead, return a small class or namedtuple instance.

Item 20: Prefer Raising Exceptions to Returning None

In [6]:
def careful_divide(a: float, b: float):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs') #retutn None 쓰는 것 대신

print(careful_divide(3,4))

0.75


Item 21: Know How Closures Interact with Variable Scope 

In [9]:
def sort_priority(values, group):
    def helper(x):#closure function 
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

1
1
1
1
1
1
1
1
[2, 3, 5, 7, 1, 4, 6, 8]


In [10]:
def sort_priority2(values, group):
    found = False
    def helper(x):#closure function 
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
print(sort_priority2(numbers, group)) #False가 출력됨. -> closure can't affect enclosing scopes by assigning variables.
print(numbers)

False
[2, 3, 5, 7, 1, 4, 6, 8]


In [11]:
def sort_priority2(values, group):
    found = False
    def helper(x):#closure function 
        if x in group:
            nonlocal found
            found = True
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
print(sort_priority2(numbers, group)) #True가 출력됨. -> nonlocal statement makes closure can affect enclosing scopes by assigning variables.
print(numbers)

True
[2, 3, 5, 7, 1, 4, 6, 8]


In [12]:
class Sorter:
    def __init__(self, group):
        self.group = group
        self.found = False
    
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sorter = Sorter(group)
numbers.sort(key=sorter) #__call__ method numbers를 순회하면서 한번씩 호출
print(numbers)
print(sorter.found)

[2, 3, 5, 7, 1, 4, 6, 8]
True


Item 22: Reduce Visual Noise with Variable Positional Arguments

In [18]:
def log(message, *values):#Variable Positional Arguments -> Tuple 형태로 받는다.
    if not values:
        print(message)
    else:
        values_str = ','.join(str(x) for x in values)
        print(f'{message}: {values_str}')

log('My numbers are', 1,2)

My numbers are: 1,2


Item 23: Provide Optional Behavior with Keyword Arguments

In [25]:
def remainer(number, divisor):
    return print(number % divisor)

remainer(20, 7)
remainer(20, divisor=7)
remainer(number=20, divisor=7)
remainer(divisor=7, number=20)
#remainer(20, number=7) 오류
#remainer(divisor=7, 20)
#remainer(number=20, 7)

6
6
6
6


In [27]:
my_kwargs = {
    'number': 20,
    'divisor': 7,
}
remainer(**my_kwargs) #딕셔너리로 받으면, 해당하는 key값을 대응하는 keyword arguments of the function에 맵핑함.

6


In [28]:
def print_parameters(**kwargs): #함수의 매개변수가 **statement를 사용하면, 딕셔너리로 전환하여 받는다.
    for key, value in kwargs.items():
        print(f'{key} = {value}')

print_parameters(alpha=1.5, beta=9, gamma=4)


alpha = 1.5
beta = 9
gamma = 4


In [31]:
def flow_rate(weight_diff, time_diff,
                period=1, units_per_kg=1):
    return ((weight_diff*units_per_kg) / time_diff) * period

weight_diff = 0.5
time_diff = 3
pounds_per_hour = flow_rate(weight_diff, time_diff, units_per_kg=2)
print(pounds_per_hour)

0.3333333333333333


Item 24: Use None and Docstrings to Specify Dynamic Default Arguments

In [33]:
from datetime import datetime

def log(message, when=None):
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')

log('Hi there!')

2022-01-05 03:15:28.436745: Hi there!


Item 25: Enforce Clarity with Keyword-Only and Positional-Only Arguments

In [44]:
def safe_division_c(number, divisor, /, *,
                      ignore_overflow, ignore_zero_division): # Keyword-only arguments are defined after single * symbol in the argument list. 
                                                              # Positional-only arguments are defined before a single / symbol in the argument list.
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

#safe_division_c(1.0, 10**500, True, False) 오류
#safe_division_c(number=1.0, divisor=10, ignore_overflow=False, ignore_zero_division=True) 오류
safe_division_c(1.0, 10*500, ignore_overflow=False, ignore_zero_division=True)

0.0002

Item 26: Define Function Decorators with functools.wraps

In [45]:
def trace(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}' f'-> {result!r}')
        return result
    return wrapper

@trace #trace(fibonacci)를 의미한다
def fibonacci(n):
    if n in (0,1):
        return n
    return (fibonacci(n-2) + fibonacci(n-1))

In [46]:
fibonacci(4)

fibonacci((0,), {}-> 0
fibonacci((1,), {}-> 1
fibonacci((2,), {}-> 1
fibonacci((1,), {}-> 1
fibonacci((0,), {}-> 0
fibonacci((1,), {}-> 1
fibonacci((2,), {}-> 1
fibonacci((3,), {}-> 2
fibonacci((4,), {}-> 3


3

In [48]:
print(fibonacci) #wrapper라 출력 -> debug시 문제가 생김
help(fibonacci)

<function trace.<locals>.wrapper at 0x000001BBCEE7E4C0>
Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



In [49]:
from functools import wraps

def trace(func):
    @wraps(func)#functools 사용
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}' f'-> {result!r}')
        return result
    return wrapper

@trace #trace(fibonacci)를 의미한다
def fibonacci(n):
    if n in (0,1):
        return n
    return (fibonacci(n-2) + fibonacci(n-1))

In [50]:
print(fibonacci) #fibonacci라 출력 -> debug시 문제없음
help(fibonacci)

<function fibonacci at 0x000001BBCEE77280>
Help on function fibonacci in module __main__:

fibonacci(n)

